# Data Cleaning and Put into SQL

In [2]:
import pandas as pd
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import bs4
import time
import random
import numpy as np

## Clean author_article.csv

In [132]:
# clean author_article
df_art_au = pd.read_csv("author_article.csv", encoding = "utf-8-sig").drop_duplicates()
df_art_au.to_csv("author_article.csv", encoding = "utf-8-sig", index = False)
df_art_au.to_csv("author_article_no_header.csv", encoding = "utf-8-sig", index = False, header = False)

## Clean article.csv

In [133]:
# select valid records with valid doi, issue format, and not duplicated
def select_valid(df):
    df_s = df[(df["doi"].str[:3] == "10.") & (df["volume"].str.contains("Supplement") == False) & 
              (df["volume"].str.contains("_Part_") == False) & 
              (df["volume"].str.contains("S1") == False) & 
              (df["volume"].str.contains("issue ") == True)].drop_duplicates(subset=["doi"]).reset_index(drop = True)
    return df_s

# convert volume to date
def conv_volume_to_date(v,freq):
    v_list = v.split(", ")
    year = v_list[0]
    issue = v_list[2].replace("issue ","")
    if freq == 4:
        return year+"-"+str(int(issue)*3-1)+"-15"
    elif freq == 6 or freq == 5:
        return year+"-"+str(int(issue)*2)+"-01"
    elif freq == 12:
        return year+"-"+issue+"-15"
    elif freq == 3:
        return year+"-"+str(int(issue)*3)+"-01"
    elif freq == 2:
        return year+"-"+str(int(issue)*6-2)+"-01"
    elif freq == 1:
        return year+"-"+str(7)+"-01"
    
# calculate issue frequency
def cal_freq(vs):
    dict = {}
    years = [int(v.split(", ")[0]) for v in vs]
    unique_years = set(years)
    issues = [int(v.split(", ")[2].replace("issue ","")) for v in vs]
    for unique_year in unique_years:
        max_issue = 0
        for i in range(len(years)):
            if unique_year == years[i]:
                if issues[i] > max_issue:
                    max_issue = issues[i]
        dict[unique_year] = max_issue

    return dict


In [134]:
# clean rfs
df_rfs = pd.read_csv("data/journals/raw_data/rfs.csv", encoding = "utf-8-sig")
df_rfs_s = select_valid(df_rfs)
freq = cal_freq(df_rfs_s["volume"])

dates = []
for idx in df_rfs_s.index:
    year = int(df_rfs_s.loc[idx,"volume"].split(", ")[0])
    dates.append(conv_volume_to_date(df_rfs_s.loc[idx,"volume"],freq[year]))

df_rfs_s["date"] = dates
df_rfs_s.head()

,journal,title,authors,volume,jel,abstract,url,doi,date
0,Review of Financial Studies,Dynamic Equilibrium with Liquidity Constraints,"['Jerome Detemple', 'Angel Serrat']","2003, vol. 16, issue 2, 597-629",[],Abstract: This article studies an intertempora...,https://econpapers.repec.org/article/ouprfinst...,10.1093/rfs/hhg003,2003-5-15
1,Review of Financial Studies,Stochastic Discount Factor Bounds with Conditi...,"['Wayne Ferson', 'Andrew F. Siegel']","2003, vol. 16, issue 2, 567-595",[],Abstract: Hansen and Jagannathan (1991) (herea...,https://econpapers.repec.org/article/ouprfinst...,10.1093/rfs/hhg004,2003-5-15
2,Review of Financial Studies,Delta-Hedged Gains and the Negative Market Vol...,"['Gurdip Bakshi', 'Nikunj Kapadia']","2003, vol. 16, issue 2, 527-566",[],Abstract: We investigate whether the volatilit...,https://econpapers.repec.org/article/ouprfinst...,10.1093/rfs/hhg002,2003-5-15
3,Review of Financial Studies,"Differences of Opinion, Short-Sales Constraint...","['Harrison Hong', 'Jeremy Stein']","2003, vol. 16, issue 2, 487-525",[],Abstract: We develop a theory of market crashe...,https://econpapers.repec.org/article/ouprfinst...,10.1093/rfs/hhg006,2003-5-15
4,Review of Financial Studies,Risk Adjustment and Trading Strategies,"['Dong-Hyun Ahn', 'Jennifer Conrad', 'Robert D...","2003, vol. 16, issue 2, 459-485",[],Abstract: We assess the profitability of momen...,https://econpapers.repec.org/article/ouprfinst...,10.1093/rfs/hhg001,2003-5-15


In [135]:
# clean res
df_res = pd.read_csv("data/journals/raw_data/res.csv", encoding = "utf-8-sig")
df_res_s = select_valid(df_res)
freq = cal_freq(df_res_s["volume"])

dates = []
for idx in df_res_s.index:
    year = int(df_res_s.loc[idx,"volume"].split(", ")[0])
    dates.append(conv_volume_to_date(df_res_s.loc[idx,"volume"],freq[year]))

df_res_s["date"] = dates
df_res_s.head()

,journal,title,authors,volume,jel,abstract,url,doi,date
0,Review of Economic Studies,"The Dynamics of Return Migration, Human Capita...","['Jerome Adda', 'Christian Dustmann', 'Joseph-...","2022, vol. 89, issue 6, 2841-2871",[],Abstract: This article develops and estimates ...,https://econpapers.repec.org/article/ouprestud...,10.1093/restud/rdac003,2022-12-01
1,Review of Economic Studies,The Impact of Car Pollution on Infant and Chil...,"['Diane Alexander', 'Hannes Schwandt']","2022, vol. 89, issue 6, 2872-2910",[],"Abstract: In 2008, Volkswagen introduced a new...",https://econpapers.repec.org/article/ouprestud...,10.1093/restud/rdac007,2022-12-01
2,Review of Economic Studies,The Welfare Effects of Transportation Infrastr...,"['Treb Allen', 'Costas Arkolakis']","2022, vol. 89, issue 6, 2911-2957",[],"Abstract: Each year in the US, hundreds of bil...",https://econpapers.repec.org/article/ouprestud...,10.1093/restud/rdac001,2022-12-01
3,Review of Economic Studies,Subjective Models of the Macroeconomy: Evidenc...,"['Peter Andrebriq', 'Carlo Pizzinelli', 'Chris...","2022, vol. 89, issue 6, 2958-2991",[],Abstract: We study people’s subjective models ...,https://econpapers.repec.org/article/ouprestud...,10.1093/restud/rdac008,2022-12-01
4,Review of Economic Studies,"Product Life Cycle, Learning, and Nominal Shocks","['David Argente', 'Chen Yeh']","2022, vol. 89, issue 6, 2992-3054",[],Abstract: This article documents a new set of ...,https://econpapers.repec.org/article/ouprestud...,10.1093/restud/rdac004,2022-12-01


In [136]:
# clean qje
df_qje = pd.read_csv("data/journals/raw_data/qje.csv", encoding = "utf-8-sig")
df_qje_s = select_valid(df_qje)
freq = cal_freq(df_qje_s["volume"])

dates = []
for idx in df_qje_s.index:
    year = int(df_qje_s.loc[idx,"volume"].split(", ")[0])
    dates.append(conv_volume_to_date(df_qje_s.loc[idx,"volume"],freq[year]))

df_qje_s["date"] = dates
df_qje_s.head()

,journal,title,authors,volume,jel,abstract,url,doi,date
0,The Quarterly Journal of Economics,Systemic Discrimination Among Large U.S. Emplo...,"['Patrick Kline', 'Evan K Rose', 'Christopher ...","2022, vol. 137, issue 4, 1963-2036",[],Abstract: We study the results of a massive na...,https://econpapers.repec.org/article/oupqjecon...,10.1093/qje/qjac024,2022-11-15
1,The Quarterly Journal of Economics,Valuing the Global Mortality Consequences of C...,"['Tamma Carleton', 'Amir Jina', 'Michael Delga...","2022, vol. 137, issue 4, 2037-2105",[],Abstract: Using 40 countries’ subnational data...,https://econpapers.repec.org/article/oupqjecon...,10.1093/qje/qjac020,2022-11-15
2,The Quarterly Journal of Economics,Reshaping Global Trade: The Immediate and Long...,['Chenzi Xu'],"2022, vol. 137, issue 4, 2107-2161",[],Abstract: I show that a disruption to the fina...,https://econpapers.repec.org/article/oupqjecon...,10.1093/qje/qjac016,2022-11-15
3,The Quarterly Journal of Economics,Sexual Harassment and Gender Inequality in the...,"['Olle Folke', 'Johanna Rickne']","2022, vol. 137, issue 4, 2163-2212",[],Abstract: We describe how sexual harassment co...,https://econpapers.repec.org/article/oupqjecon...,10.1093/qje/qjac018,2022-11-15
4,The Quarterly Journal of Economics,The Quality and Efficiency of Public and Priva...,"['Daniel Knutsson', 'Björn Tyrefors']","2022, vol. 137, issue 4, 2213-2262",[],Abstract: Economic theory predicts that outsou...,https://econpapers.repec.org/article/oupqjecon...,10.1093/qje/qjac014,2022-11-15


In [137]:
# clean jpe
df_jpe = pd.read_csv("data/journals/raw_data/jpe.csv", encoding = "utf-8-sig")
df_jpe["doi"] = df_jpe["doi"].apply(lambda x: x.replace("http://dx.doi.org/",""))
df_jpe_s = select_valid(df_jpe)
freq = cal_freq(df_jpe_s["volume"])

dates = []
for idx in df_jpe_s.index:
    year = int(df_jpe_s.loc[idx,"volume"].split(", ")[0])
    dates.append(conv_volume_to_date(df_jpe_s.loc[idx,"volume"],freq[year]))

df_jpe_s["date"] = dates

# modify date for 2023
for idx in df_jpe_s.index:
    year = int(df_jpe_s.loc[idx,"volume"].split(", ")[0])
    if year == 2023:
        df_jpe_s.loc[idx, "date"] = conv_volume_to_date(df_jpe_s.loc[idx,"volume"],12)

df_jpe_s.head()

,journal,title,authors,volume,jel,abstract,url,doi,date
0,Journal of Political Economy,Collective Brand Reputation,"['Volker Nocke', 'Roland Strausz']","2023, vol. 131, issue 1, 1 - 58",[],Abstract: We develop a theory of collective br...,https://econpapers.repec.org/article/ucpjpolec...,10.1086/720986,2023-1-15
1,Journal of Political Economy,The Mechanics of the Industrial Revolution,"['Morgan Kelly', 'Joel Mokyr', 'Cormac Ó Gráda']","2023, vol. 131, issue 1, 59 - 94",[],Abstract: Although there are many competing ex...,https://econpapers.repec.org/article/ucpjpolec...,10.1086/720890,2023-1-15
2,Journal of Political Economy,Optimal Cooperative Taxation in the Global Eco...,"['V. V. Chari', 'Juan Pablo Nicolini', 'Pedro ...","2023, vol. 131, issue 1, 95 - 130",[],Abstract: How should countries cooperate in se...,https://econpapers.repec.org/article/ucpjpolec...,10.1086/720889,2023-1-15
3,Journal of Political Economy,Personalized Pricing and Consumer Welfare,"['Jean-Pierre Dubé', 'Sanjog Misra']","2023, vol. 131, issue 1, 131 - 189",[],Abstract: We study the welfare implications of...,https://econpapers.repec.org/article/ucpjpolec...,10.1086/720793,2023-1-15
4,Journal of Political Economy,A Semistructural Methodology for Policy Counte...,['Martin Beraja'],"2023, vol. 131, issue 1, 190 - 201",[],Abstract: I propose a methodology for construc...,https://econpapers.repec.org/article/ucpjpolec...,10.1086/720982,2023-1-15


In [ ]:
for v in df_jpe_s.volume:
    print(v)

In [138]:
# concatenate journals
df = pd.concat([df_rfs_s,df_res_s,df_qje_s,df_jpe_s]).reset_index(drop=True)
len(df)

14201

In [139]:
df.nunique()

journal         4
title       14047
authors     10586
volume      14201
jel           624
abstract     8959
url         14201
doi         14201
date         1057
dtype: int64

In [140]:
df_selected = df[["doi","journal","volume","date","title","abstract","url"]]
df_selected.head()

,doi,journal,volume,date,title,abstract,url
0,10.1093/rfs/hhg003,Review of Financial Studies,"2003, vol. 16, issue 2, 597-629",2003-5-15,Dynamic Equilibrium with Liquidity Constraints,Abstract: This article studies an intertempora...,https://econpapers.repec.org/article/ouprfinst...
1,10.1093/rfs/hhg004,Review of Financial Studies,"2003, vol. 16, issue 2, 567-595",2003-5-15,Stochastic Discount Factor Bounds with Conditi...,Abstract: Hansen and Jagannathan (1991) (herea...,https://econpapers.repec.org/article/ouprfinst...
2,10.1093/rfs/hhg002,Review of Financial Studies,"2003, vol. 16, issue 2, 527-566",2003-5-15,Delta-Hedged Gains and the Negative Market Vol...,Abstract: We investigate whether the volatilit...,https://econpapers.repec.org/article/ouprfinst...
3,10.1093/rfs/hhg006,Review of Financial Studies,"2003, vol. 16, issue 2, 487-525",2003-5-15,"Differences of Opinion, Short-Sales Constraint...",Abstract: We develop a theory of market crashe...,https://econpapers.repec.org/article/ouprfinst...
4,10.1093/rfs/hhg001,Review of Financial Studies,"2003, vol. 16, issue 2, 459-485",2003-5-15,Risk Adjustment and Trading Strategies,Abstract: We assess the profitability of momen...,https://econpapers.repec.org/article/ouprfinst...


In [141]:
# save article
df_selected.to_csv("article.csv", index = False, encoding = "utf-8-sig")
df_selected.to_csv("article_no_header.csv", index = False, header = False, encoding = "utf-8-sig")

## Clean and update author.csv, email_affiliation.csv, affiliation.csv

In [151]:
# clean author
df_email = pd.read_csv("email_affiliation.csv", encoding = "utf-8-sig").replace(np.NaN,"")
emaildict = df_email.to_dict("list")
df_au = pd.read_csv("author.csv", encoding = "utf-8-sig")

In [147]:
# open chrome service
chromedriver_path = "D:/chromedriver.exe"
s = Service(chromedriver_path)
driver = webdriver.Chrome(service=s)

In [152]:
base_url = "https://scholar.google.com/scholar?hl=zh-CN&q="
start_idx = 1
for i, e in enumerate(sorted(df_au["emailsuffix"].unique())[start_idx:]):
    if e not in list(df_email.subemail):
        driver.get(base_url+e)
        driver.refresh()
        soup = bs4.BeautifulSoup(driver.page_source, "html.parser")
        try:
            h3 = soup.find("div", class_ = "gs_r").h3
            if h3.text == "以下机构的用户个人学术档案：":
                div = soup.find("div", class_ = "gs_r").div
                name = div.a.text
                email = div.span.text.replace(" - ", "")
                time.sleep(random.randrange(5,10))
            else:
                raise ValueError("x")
        except:
            try:
                h2 = soup.find("div", class_ = "gs_r").h2
                if "显示的是以下查询字词的结果：" in h2.text.strip():
                    driver.get("https://scholar.google.com"+soup.find("div", class_ = "gs_r").div.a.get("href"))
                    driver.refresh()
                    soup = bs4.BeautifulSoup(driver.page_source, "html.parser")
                    div = soup.find("div", class_ = "gs_r").div
                    name = div.a.text
                    email = div.span.text.replace(" - ", "")
                    time.sleep(random.randrange(5,10))
                else:
                    raise ValueError("x")
            except:
                driver.get(base_url+".".join(e.split(".")[1:]))
                driver.refresh()
                soup = bs4.BeautifulSoup(driver.page_source, "html.parser")
                try:
                    h3 = soup.find("div", class_ = "gs_r").h3
                    if h3.text == "以下机构的用户个人学术档案：":
                        div = soup.find("div", class_ = "gs_r").div
                        name = div.a.text
                        email = div.span.text.replace(" - ", "")
                        time.sleep(random.randrange(5,10))
                    else:
                        raise ValueError("x")
                except:
                    try:
                        h2 = soup.find("div", class_ = "gs_r").h2
                        if "显示的是以下查询字词的结果：" in h2.text.strip():
                            driver.get("https://scholar.google.com"+soup.find("div", class_ = "gs_r").div.a.get("href"))
                            driver.refresh()
                            soup = bs4.BeautifulSoup(driver.page_source, "html.parser")
                            div = soup.find("div", class_ = "gs_r").div
                            name = div.a.text
                            email = div.span.text.replace(" - ", "")
                            time.sleep(random.randrange(5,10))
                        else:
                            raise ValueError("x")
                    except:
                        try:
                            if soup.find("h1").text == "请进行人机身份验证":
                                print("Blocked!!!")
                                break
                            else:
                                name = ""
                                email = ""
                            time.sleep(random.randrange(5,10))
                        except:
                            name = ""
                            email = ""
                            time.sleep(random.randrange(5,10))
        # append info
        try:
            affid = "".join([w[0] for w in name.lower().split(" ")])+"."+email
        except:
            affid = ""

        if "PDF" in affid:
            name = ""
            email = ""
            affid = ""

        emaildict["name"].append(name)
        emaildict["email"].append(email)
        emaildict["subemail"].append(e)
        emaildict["affiliationid"].append(affid)
        print(i+start_idx, e, name, email, affid)

85 bot.or.th   
228 easternbank.com   


In [153]:
len(emaildict["name"])

1048

In [154]:
df_email = pd.DataFrame(emaildict).drop_duplicates(subset=["subemail"],keep="last")
len(df_email)

1048

In [155]:
df_email.nunique()

name              578
email             578
subemail         1048
affiliationid     578
dtype: int64

In [156]:
df_aff = df_email[["affiliationid","name","email"]][df_email["affiliationid"]!=""].drop_duplicates()
len(df_aff)

577

In [157]:
# add affiliationid to author
for idx in df_au.index:
    try:
        df_au.loc[idx, "affiliationid"] = df_email.set_index("subemail").loc[df_au.loc[idx, "emailsuffix"],"affiliationid"]
    except:
        df_au.loc[idx, "affiliationid"] = ""

In [158]:
df_email.to_csv("email_affiliation.csv", index = False, encoding = "utf-8-sig")
df_email.to_csv("email_affiliation_no_header.csv", index = False, header = False, encoding = "utf-8-sig")
df_aff.to_csv("affiliation.csv", index = False, encoding = "utf-8-sig")
df_aff.to_csv("affiliation_no_header.csv", index = False, header = False, encoding = "utf-8-sig")
df_au.to_csv("author.csv", index = False, encoding = "utf-8-sig")
df_au.to_csv("author_no_header.csv", index = False, header = False, encoding = "utf-8-sig")

## Database

In [159]:
# use this line to run the script directly in Jupyter
! sqlite3 econtop.db < create-econtop.sql

In [160]:
conn = sqlite3.connect("econtop.db")
cur = conn.cursor()

### University Level

In [170]:
# select university level of publications
q = """
    SELECT af.affiliationid, af.name, art.journal, aa.doi
    FROM author_article AS aa JOIN author AS au JOIN article AS art JOIN affiliation AS af
    ON aa.authorid = au.authorid AND
    aa.doi = art.doi AND
    au.affiliationid = af.affiliationid
    """

# q = " SELECT authorid FROM author"
df_af_art = pd.read_sql_query(q, conn).set_index("doi")
df_af_art.head()

,affiliationid,name,journal
doi,,,
10.1093/rfs/hhg003,bu.bu.edu,Boston University,Review of Financial Studies
10.1093/rfs/hhg004,uosc.usc.edu,University of Southern California,Review of Financial Studies
10.1093/rfs/hhg002,tu.temple.edu,Temple University,Review of Financial Studies
10.1093/rfs/hhg002,uoma.umass.edu,"University of Massachusetts, Amherst",Review of Financial Studies
10.1093/rfs/hhg006,cu.columbia.edu,Columbia University,Review of Financial Studies


In [ ]:
# show top appearance


In [166]:
id_pair = {}
name_pair = {}

for doi in df_af_art.index:
    try:
        df_af_art.loc[doi][0]
    except:
        df_af_art_new = df_af_art.loc[doi].sort_values("affiliationid")
        length = len(df_af_art_new)
        for i in range(length-1):
            for j in range(length-1-i):
                try:
                    id_pair[df_af_art_new["affiliationid"][i]+"+"+df_af_art_new["affiliationid"][i+j+1]] += 1
                    name_pair[df_af_art_new["name"][i]+"+"+df_af_art_new["name"][i+j+1]] += 1
                except:
                    id_pair[df_af_art_new["affiliationid"][i]+"+"+df_af_art_new["affiliationid"][i+j+1]] = 0
                    name_pair[df_af_art_new["name"][i]+"+"+df_af_art_new["name"][i+j+1]] = 0

In [167]:
name_pair_sorted = sorted(name_pair.items(), key=lambda x:x[1], reverse = True)
name_pair_sorted

[('Harvard University+Harvard University', 125),
 ('Harvard University+Massachusetts Institute of Technology', 110),
 ('Massachusetts Institute of Technology+Massachusetts Institute of Technology',
  73),
 ('University of California, Berkeley+University of California, Berkeley', 68),
 ('University of Chicago+University of Chicago', 62),
 ('Massachusetts Institute of Technology+Stanford University', 62),
 ('Stanford University+Stanford University', 60),
 ('Massachusetts Institute of Technology+University of Chicago', 59),
 ('Harvard University+Stanford University', 56),
 ('Columbia University+Columbia University', 49),
 ('Northwestern University+University of Chicago', 48),
 ('Harvard University+University of California, Berkeley', 44),
 ('Massachusetts Institute of Technology+Northwestern University', 40),
 ('Harvard University+University of Chicago', 39),
 ('University of Chicago+Yale University', 39),
 ('University of Chicago+University of California, Berkeley', 38),
 ('Columbia Univ

In [131]:
conn.close()